# Introduction

This notebook evaluates the best model in notebook `04_modelling` on the hold-out dataset.

# Libraries

In [4]:
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import pandas as pd
import numpy as np

# Data

The data used to do the evaluation must be in the same format as the one used to train the model i.e. a set of summary statistics per trip. These features can be computed using the `03_feature_engineering` notebook.
For reference, the following cell shows the columns that need to be present in the hold-out dataset:

In [7]:
col_names = ['bookingID',
 'trip_length',
 'mean_Speed',
 'sd_Speed',
 'min_Speed',
 'max_Speed',
 'mean_mag_acc',
 'sd_mag_acc',
 'min_mag_acc',
 'max_mag_acc',
 'mean_mag_gyro',
 'sd_mag_gyro',
 'min_mag_gyro',
 'max_mag_gyro',
 'Speed_quantiles_0',
 'mag_acc_quantiles_0',
 'mag_gyro_quantiles_0',
 'Speed_quantiles_1',
 'mag_acc_quantiles_1',
 'mag_gyro_quantiles_1',
 'Speed_quantiles_2',
 'mag_acc_quantiles_2',
 'mag_gyro_quantiles_2',
 'Speed_quantiles_3',
 'mag_acc_quantiles_3',
 'mag_gyro_quantiles_3',
 'Speed_quantiles_4',
 'mag_acc_quantiles_4',
 'mag_gyro_quantiles_4',
 'label']

n = len(col_names)

hold_out_df = pd.DataFrame(np.random.rand(10, n), columns=col_names)
hold_out_df['label'] = np.random.binomial(1, 0.5, size=(10))
hold_out_df['bookingID'] = np.random.randint(1000, 20000, size=(10))

hold_out_df = spark.createDataFrame(hold_out_df)

display(hold_out_df)

bookingID,trip_length,mean_Speed,sd_Speed,min_Speed,max_Speed,mean_mag_acc,sd_mag_acc,min_mag_acc,max_mag_acc,mean_mag_gyro,sd_mag_gyro,min_mag_gyro,max_mag_gyro,Speed_quantiles_0,mag_acc_quantiles_0,mag_gyro_quantiles_0,Speed_quantiles_1,mag_acc_quantiles_1,mag_gyro_quantiles_1,Speed_quantiles_2,mag_acc_quantiles_2,mag_gyro_quantiles_2,Speed_quantiles_3,mag_acc_quantiles_3,mag_gyro_quantiles_3,Speed_quantiles_4,mag_acc_quantiles_4,mag_gyro_quantiles_4,label
15290,0.7704865804642453,0.8769988614523183,0.9533830154551796,0.7388362925447499,0.5261368927267606,0.3833302192180623,0.27935972060969516,0.4561212226802598,0.7445342215266202,0.5317171999425669,0.5099258753780906,0.7361743495790696,0.18472966343192954,0.5624939872497654,0.22374667450249364,0.0678203120263361,0.7585907093132831,0.072763516800738,0.4814634428409613,0.7694746678721542,0.9543703530679604,0.6520015490261077,0.34069053261812565,0.3257917035325413,0.20477884688243342,0.9964555054841421,0.9677377405864747,0.06769162426053843,1
4706,0.02439961586363093,0.2701742303266408,0.5641055702421844,0.5501547843010166,0.6231018392144877,0.9769373459584679,0.8272495462656171,0.09399162847335096,0.5647902048930948,0.6959049103085899,0.5195078776002879,0.9480483271455915,0.07320706254930864,0.25314242331156933,0.9383331338840137,0.34955893603764043,0.8372870656996263,0.4771701784862845,0.9409993135400045,0.2185944289780889,0.25240819277527315,0.4477723084207652,0.9865442934125243,0.4433606148552016,0.4331561424874433,0.32925220013868717,0.4855476407607039,0.2898231731240559,1
3376,0.3239652812513788,0.6529237811927086,0.23945868015030713,0.31118699126273675,0.8508019007951538,0.3475953473393746,0.9389689156703589,0.3781455886709856,0.506499275268257,0.8057844704888272,0.8683566402366958,0.7595201837541233,0.1706945340185423,0.006970716003414368,0.8639161286539048,0.02483119959953739,0.05994487480460631,0.27273458844890575,0.20484394937879846,0.9032333219898057,0.9362333184588878,0.9083939539203312,0.5021246939372643,0.8423676517008849,0.5114161421107265,0.980961472377236,0.8223264075416478,0.6121376607461715,1
11212,0.7703827138579988,0.5219960578068827,0.9303808620780519,0.4465375490425699,0.21471634313504895,0.645720701200869,0.7439569690145142,0.40654164590822794,0.8488308348902119,0.8121314001849806,0.4911155215064609,0.08653008182239763,0.9506513520825848,0.14093073577146908,0.6240044830024685,0.8238149540727769,0.19310573977474443,0.7472715557539384,0.5752818714045443,0.40310976777357044,0.7766218210098996,0.7053095188156473,0.7488529669974808,0.07965055806764143,0.7179020826137628,0.8259240418290934,0.7853336390265799,0.8043919824227284,0
16938,0.9364160693654497,0.2615265626091916,0.18453144720012737,0.7339076621888464,0.806858499340016,0.5886498964482918,0.2839091385870517,0.7127227877365069,0.42275325727620117,0.4560883786354958,0.7377154220348532,0.6707129437689392,0.049022645739286386,0.14208638865869694,0.5832007448780698,0.0035432391864206325,0.6679212533733506,0.36644446492992566,0.11485442025442472,0.2005011724643453,0.9971878177566026,0.9164488152099345,0.4331070810892077,0.09615242925795897,0.004376163280839451,0.09501590196745702,0.7189541346765833,0.012632066096986194,0
17984,0.4345242643441368,0.7284718917450412,0.07560470184983492,0.11002607963843047,0.2702517148332011,0.8205419132835963,0.6661850622320855,0.3938481517101703,0.8252282445560021,0.7919191968197237,0.18221978469329247,0.8636812824548779,0.605272589175943,0.5360462849363621,0.028432413802875756,0.7612759920635436,0.3515582132184024,0.8454531996417107,0.38384499348419554,0.9501735997009342,0.2885623802342273,0.6517192622899115,0.588463456484222,0.06200866077473466,0.5876508158714977,0.6288202064579107,0.152136543089191,0.8027852178057535,1
14444,0.1917775012558458,0.977560400659408,0.6352734628020775,1.2759698278330678E-4,0.7888570377638557,0.6360413099318829,0.24919451274656212,0.5629770399297443,0.6403614107951222,0.8191780097943502,0.6058618881740904,0.9251674039570972,0.4679127541914011,

In [8]:
# uncomment this once you have generated the features for the hold-out dataset.
# refer to the "03_feature_engineering" notebook for details

# HOLD_OUT_PATH = "PATH/TO/HOLD_OUT.parquet"
# hold_out_df = spark.read.parquet(HOLD_OUT_PATH)

# Model

Define path to the saved model:

In [11]:
MODEL_DIR = 'dbfs:/msh/grab/model/best_model'

Load the best model:

In [13]:
model = PipelineModel.load(MODEL_DIR)

Create an evaluator:

In [15]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Evaluation

Compute the model's AUC-ROC curve on the hold-out dataset:

In [18]:
hold_out_auc = evaluator.evaluate(model.transform(hold_out_df))
print(f'The AUC-ROC curve on the hold-out dataset is {hold_out_auc:.4f}.')

The AUC-ROC curve on the hold-out dataset is 0.7917.